In [42]:

import openai
import pandas as pd
import json
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')


In [12]:
conv_data = pd.read_excel('data/conversations_for_llama_6656.xlsx')
conv_data = conv_data[['id','conversation_id','proc_text']]
conv_data['first_role'] = conv_data['proc_text'].apply(lambda x : "Teacher" if x.startswith('teacher') else 'Student')
conv_data['second_role'] = conv_data['proc_text'].apply(lambda x : "Student" if x.startswith('teacher') else 'Teacher')
conv_data = conv_data.sort_values(['conversation_id','id']).reset_index(drop=True)
conv_data

,id,conversation_id,proc_text,first_role,second_role
0,45939,7,"teacher: hi there <student>, all ok? student:...",Teacher,Student
1,45940,7,"teacher: yeah i'm good thanks, just been for a...",Teacher,Student
2,45941,7,teacher: yeah? what did you do? i'll come back...,Teacher,Student
3,45942,7,teacher: ok i see ...too bad about the running...,Teacher,Student
4,45943,7,teacher: yes i realise it's not easy really! i...,Teacher,Student
...,...,...,...,...,...
6651,56386,258,teacher: but 'to loan' means 'to lend' studen...,Teacher,Student
6652,56387,258,"teacher: banks loan money yes, exactly, it wou...",Teacher,Student
6653,56388,258,"teacher: ooh, we're out of time! student: or ...",Teacher,Student
6654,56389,258,"teacher: yes, absolutely! student: 13:59 on m...",Teacher,Student


In [18]:
conv_data

,id,conversation_id,proc_text,first_role,second_role
0,45939,7,"teacher: hi there <student>, all ok? student:...",Teacher,Student
1,45940,7,"teacher: yeah i'm good thanks, just been for a...",Teacher,Student
2,45941,7,teacher: yeah? what did you do? i'll come back...,Teacher,Student
3,45942,7,teacher: ok i see ...too bad about the running...,Teacher,Student
4,45943,7,teacher: yes i realise it's not easy really! i...,Teacher,Student
...,...,...,...,...,...
6651,56386,258,teacher: but 'to loan' means 'to lend' studen...,Teacher,Student
6652,56387,258,"teacher: banks loan money yes, exactly, it wou...",Teacher,Student
6653,56388,258,"teacher: ooh, we're out of time! student: or ...",Teacher,Student
6654,56389,258,"teacher: yes, absolutely! student: 13:59 on m...",Teacher,Student


In [31]:

instructions = """
Please rate a student teacher conversation based on how interesting they feel. Think like a human.Provide a rating between 0 to 4. O being boring. 4 being most interesting. It is a conversation between a student (2nd language English) learning English from a teacher. Assume the role of the student based on the conversation. Please give the Rating as an integer value in this format: 
'INT: {integer rating from 0 to 4},
EXP_INT:{integer rating from 0 to 4},
REASON: {justification for rating}.'
Please keep the previous conversation in mind. 
""" 


responses = []
prev_id = 0

for idx, conv in conv_data.iterrows():

    first_role = conv['first_role']
    second_role = conv['second_role']
    conv_text = conv['proc_text']
    conv_turnid = conv['id']


    ## beginning of a conversation
    if conv_turnid-prev_id >1:
        completion = openai.ChatCompletion.create(
            model="gpt-4o",
            max_tokens=4096,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": f"Assume the role of {first_role} and rate {second_role}. Rate this conversation {conv_text} and give the output in a json format." }
            ]
        )
        assistant_msg = completion.choices[0].message.content
        responses.append(assistant_msg)


    else: ## continued conversation
        completion = openai.ChatCompletion.create(
            model="gpt-4o",
            max_tokens=4096,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": instructions},
                {"role": "user", "content": f"Assume the role of {first_role} and rate {second_role}. Rate this conversation {prev_conv} and give the output in a json format." },
                {"role": "assistant", "content":  prev_assistant_msg},
                {"role": "user", "content": f"Assume the role of {first_role} and rate {second_role}. Rate this conversation {conv_text} and give the output in a json format." },
            ]
        )
        assistant_msg = completion.choices[0].message.content
        responses.append(assistant_msg)

    prev_id = conv_turnid
    prev_conv = conv_text
    prev_assistant_msg =assistant_msg
   

In [34]:
len(responses), len(set(responses))

(6656, 6656)

In [37]:
conv_data['gpt4o_responses'] = responses

In [40]:
conv_data[['id', 'conversation_id', 'proc_text', 'gpt4o_responses']].to_excel('llm_responses/gpt4o_ratings.xlsx', index=False)